In [5]:
import openai
import os

from dotenv import find_dotenv, load_dotenv
from typing import List, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, MessagesState, StateGraph, START
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

In [3]:
class UserProfile(TypedDict):
    """User profile schema with typed fields.
    """
    user_name: str
    interests: List[str]

In [4]:
user_profile: UserProfile = {
    "user_name": "Yoona",
    "interests": ["k-pop", "AI", "fashion"]
}
user_profile

{'user_name': 'Yoona', 'interests': ['k-pop', 'AI', 'fashion']}

In [6]:
in_memory_store = InMemoryStore()

user_id = "1"
namespace = (user_id, "memory")
key = "user_profile"
value = user_profile
in_memory_store.put(namespace, key, value)

In [8]:
profile = in_memory_store.get(namespace, "user_profile")
profile.value

{'user_name': 'Yoona', 'interests': ['k-pop', 'AI', 'fashion']}

In [9]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
model_with_structure = llm.with_structured_output(UserProfile)

# invoke the model to produce structured output that matches the schema
structured_output = model_with_structure.invoke([HumanMessage("My name is Yoona, i love super junior.")])
structured_output


{'user_name': 'Yoona', 'interests': ['super junior']}

In [12]:
MODEL_SYSTEM_MESSAGE = """
    You are a helpful assistant with memory that provides information about the user. 
    If you have memory for this user, use it to personalize your responses.
    Here is the memory (it may be empty): {memory}
"""

CREATE_MEMORY_INSTRUCTION = """
    Create or update a user profile memory based on the user's chat history. 
    This will be saved for long-term memory. If there is an existing memory, simply update it. 
    Here is the existing memory (it may be empty): {memory}
"""

def call_model(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Load memory from the store and use it to personalize the chatbot's response.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("memory", user_id)
    existing_memory = store.get(namespace, "user_memory")

    if existing_memory and existing_memory.value:
        memory_dict = existing_memory.value
        formatted_memory = (
            f"Name: {memory_dict.get('user_name', 'Unknown')}\n"
            f"Interests: {', '.join(memory_dict.get('interests', []))}"
        )
    else:
        formatted_memory = None

    system_msg = MODEL_SYSTEM_MESSAGE.format(memory=formatted_memory)

    response = llm.invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": response}

def write_memory(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Reflect on the chat history and save a memory to the store.
    """
    
    user_id = config["configurable"]["user_id"]
    namespace = ("memory", user_id)
    existing_memory = store.get(namespace, "user_memory")

    if existing_memory and existing_memory.value:
        memory_dict = existing_memory.value
        formatted_memory = (
            f"Name: {memory_dict.get('user_name', 'Unknown')}\n"
            f"Interests: {', '.join(memory_dict.get('interests', []))}"
        )
    else:
        formatted_memory = None
        
    system_msg = CREATE_MEMORY_INSTRUCTION.format(memory=formatted_memory)

    new_memory = model_with_structure.invoke([SystemMessage(content=system_msg)]+state['messages'])

    key = "user_memory"
    store.put(namespace, key, new_memory)

In [14]:
graph = StateGraph(MessagesState)
graph.add_node("call_model", call_model)
graph.add_node("write_memory", write_memory)
graph.add_edge(START, "call_model")
graph.add_edge("call_model", "write_memory")
graph.add_edge("write_memory", END)

across_thread_memory = InMemoryStore()
within_thread_memory = MemorySaver()
graph = graph.compile(checkpointer=within_thread_memory, store=across_thread_memory)

In [15]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}

input_messages = [HumanMessage(content="Hi, my name is Yoona, i love 2nd generation k-pop group and AI enthusiast.")]
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

Hi, my name is Yoona, i love 2nd generation k-pop group and AI enthusiast.
================================== Ai Message ==================================

Nice to meet you, Yoona! It's great to hear that you love 2nd generation K-pop groups and are an AI enthusiast. If you have any questions about K-pop or AI, feel free to ask!


In [16]:
user_id = "1"
namespace = ("memory", user_id)
existing_memory = across_thread_memory.get(namespace, "user_memory")
existing_memory.value

{'user_name': 'Yoona',
 'interests': ['2nd generation K-pop groups', 'AI enthusiast']}